In [1]:
import numpy as np
# Peng-Robinson EOS - GENERAL
R = 8.314462618  # J/mol/K

def peng_robinson(T, P, Tc, Pc, omega, M):
    """
    T  : temperature (K)
    P  : pressure (Pa)
    Tc : critical temperature (K)
    Pc : critical pressure (Pa)
    omega : acentric factor
    M  : molar mass (kg/mol)
    """

    # --- PR parameters ---
    k = 0.37464 + 1.54226*omega - 0.26992*omega**2
    Tr = T / Tc
    alpha = (1 + k*(1 - np.sqrt(Tr)))**2

    a = 0.45724 * (R**2 * Tc**2) / Pc
    b = 0.07780 * (R * Tc) / Pc

    A = a * alpha * P / (R**2 * T**2)
    B = b * P / (R * T)

    # --- Cubic equation coefficients ---
    coeffs = [
        1,
        -(1 - B),
        A - 3*B**2 - 2*B,
        -(A*B - B**2 - B**3)
    ]

    # --- Solve cubic equation ---
    Z_roots = np.roots(coeffs)
    Z_real = np.real(Z_roots[np.isreal(Z_roots)])

    Z_gas = max(Z_real)
    Z_liq = min(Z_real)

    # --- Molar volume ---
    Vm_gas = Z_gas * R * T / P
    Vm_liq = Z_liq * R * T / P

    # --- Density ---
    rho_gas = M / Vm_gas
    rho_liq = M / Vm_liq

    return {
        "Z_roots": Z_real,
        "Z_gas": Z_gas,
        "Z_liq": Z_liq,
        "rho_gas": rho_gas,
        "rho_liq": rho_liq
    }

# ==============================
# EXAMPLE: Nitrogen N2
# ==============================

T = 273.15          # K
P = 50e5            # Pa (50 bar)
Tc = 126.2          # K
Pc = 33.98e5        # Pa
omega = 0.037
M = 28.0134e-3      # kg/mol

results = peng_robinson(T, P, Tc, Pc, omega, M)

print("Z roots :", results["Z_roots"])
print("Z (gas) :", results["Z_gas"])
print("Z (liq) :", results["Z_liq"])
print("Gas density (kg/m3):", results["rho_gas"])
print("Liquid density (kg/m3):", results["rho_liq"])


Z roots : [0.97274046]
Z (gas) : 0.9727404571470275
Z (liq) : 0.9727404571470275
Gas density (kg/m3): 63.402073027972946
Liquid density (kg/m3): 63.402073027972946


In [2]:
import numpy as np

# 1. Données de l'énoncé
T_degC = 105
P_MPa = 22.5
R = 8.31446  # J/(mol.K) ou kPa.L/(mol.K)

# Composition et propriétés (C1, C2, C3, nC4)
y = np.array([0.9443, 0.0372, 0.0132, 0.0053])
M = np.array([16.0, 30.1, 44.1, 58.1])
Tc = np.array([190.6, 305.4, 369.8, 425.2])
Pc = np.array([4.60, 4.88, 4.25, 3.79])
omega = np.array([0.011, 0.099, 0.152, 0.199]) # Facteurs acentriques

# Conversion des unités
T = T_degC + 273.15  # Kelvin
P = P_MPa * 10       # bar (plus simple pour les équations d'état)
Pc_bar = Pc * 10

# --- a) Propriétés pseudo-critiques (Règles de Kay) ---
M_mel = np.sum(y * M)
Tpc = np.sum(y * Tc)
Ppc = np.sum(y * Pc)
w_mel = np.sum(y * omega)

print(f"--- Propriétés du mélange ---")
print(f"Masse molaire apparente : {M_mel:.2f} kg/kmole")
print(f"T pseudo-critique : {Tpc:.2f} K")
print(f"P pseudo-critique : {Ppc:.2f} MPa")

# --- b) Équation d'état de Peng-Robinson ---
Tr = T / Tpc
Pr = P / (Ppc * 10)

# Paramètres PR
k = 0.37464 + 1.54226 * w_mel - 0.26992 * w_mel**2
alpha = (1 + k * (1 - np.sqrt(Tr)))**2

A = 0.45724 * (Pr / Tr**2) * alpha
B = 0.07780 * (Pr / Tr)

# Coefficients du polynôme Z^3 + c2*Z^2 + c1*Z + c0 = 0
c2 = -(1 - B)
c1 = A - 3*B**2 - 2*B
c0 = -(A*B - B**2 - B**3)

# Résolution des racines
racines = np.roots([1, c2, c1, c0])
# On garde la racine réelle la plus grande (phase gaz)
Z = np.real(racines[np.isreal(racines)].max())

# Calcul de la masse volumique (rho = P*M / Z*R*T)
# P en Pa (P_MPa * 1e6), M en kg/mol (M_mel / 1000)
rho = (P_MPa * 1e6 * (M_mel/1000)) / (Z * 8.314 * T)

print(f"\n--- Résultats Peng-Robinson ---")
print(f"Z (Coefficient de compressibilité) : {Z:.4f}")
print(f"Masse volumique (rho) : {rho:.2f} kg/m3")

--- Propriétés du mélange ---
Masse molaire apparente : 17.12 kg/kmole
T pseudo-critique : 198.48 K
P pseudo-critique : 4.60 MPa

--- Résultats Peng-Robinson ---
Z (Coefficient de compressibilité) : 0.9296
Masse volumique (rho) : 131.80 kg/m3


In [5]:
%pip install pyvista
import pyvista as pv
import numpy as np

def create_wellbore_viz():
    # 1. Generate Wellpath Data (MD, TVD, North, East)
    # Creating a simple "J-shaped" well
    depths = np.linspace(0, 5000, 100)
    east = np.zeros_like(depths)
    north = np.zeros_like(depths)
    
    # Add a kick-off point at 2000ft
    kick_off_idx = 40
    east[kick_off_idx:] = np.linspace(0, 500, len(depths) - kick_off_idx)
    
    # Combine into coordinates [X, Y, Z]
    # Note: Z is negative for depth
    points = np.column_stack((east, north, -depths))

    # 2. Create the Casing (Tube)
    # Create a polyline from points
    path = pv.Polyline(points)
    # Turn the line into a 3D tube (casing)
    casing = path.tube(radius=20)

    # 3. Create the Wellhead
    # Represented as a cylinder at the surface (0,0,0)
    wellhead = pv.Cylinder(center=(0, 0, 10), direction=(0, 0, 1), radius=40, height=50)

    # 4. Setup Plotter
    plotter = pv.Plotter()
    plotter.add_mesh(casing, color="silver", specular=0.5, label="Casing")
    plotter.add_mesh(wellhead, color="darkred", label="Wellhead")
    
    # Add a ground plane for context
    ground = pv.Plane(center=(250, 0, 0), direction=(0, 0, 1), x_size=1000, y_size=1000)
    plotter.add_mesh(ground, color="brown", opacity=0.3)

    # Aesthetics
    plotter.add_legend()
    plotter.add_axes()
    plotter.show_grid()
    plotter.background_color = "white"
    
    print("Opening 3D viewer...")
    plotter.show(jupyter=True)

if __name__ == "__main__":
    create_wellbore_viz()

AttributeError: module 'pyvista' has no attribute 'Polyline'